# Análisis textual

In [2]:
# Librerías a utilizar

# Análisis de datos
import pandas as pd
pd.options.display.max_columns = None
import re
import numpy as np

# Análisis de contenido 
import spacy
# !python -m spacy download es_core_news_md  # Modelo mediano (recomendado) (de preferencia instalar en terminal)
# Carga el modelo en español
nlp = spacy.load("es_core_news_md")
from collections import Counter
from transformers import pipeline


In [20]:
# Cargando las cuatro bases de datos
# La Voz del Sur
#voz = pd.read_csv('scraped data/articulos_lavozdelsur.csv')
# Letra Fría
#letra = pd.read_csv('scraped data/letra_fria_200_20250414_120843.csv')

# Al Calor Político
calor = pd.read_csv('scraped data/noticias_alcalorpolitico.csv')
# Plumas Libres
#plumas = pd.read_csv('scraped data/plumas_libres_sociedad.csv')

In [24]:
calor

,titulo,fecha,autor,contenido,url,longitud,lugar,fecha_limpia
0,Magistrada Mónica Soto asume como presidenta d...,Ciudad de México 01/01/2024,Desconocido,La magistrada Mónica Soto inició este 1° de en...,https://www.alcalorpolitico.com/informacion/ma...,2173,Ciudad de México,01/01/2024
1,Instructora de Orizaba lamenta “desvalorizació...,"Orizaba, Ver. 01/01/2024",Desconocido,Aunque a través de los talleres del DIF se bus...,https://www.alcalorpolitico.com/informacion/in...,1494,"Orizaba, Ver",01/01/2024
2,"Hacen “sándwich” a auto particular, en autopis...",Fortín de las Flores 01/01/2024,Desconocido,Daños materiales por varios miles de pesos y l...,https://www.alcalorpolitico.com/informacion/ha...,1050,Fortín de las Flores,01/01/2024
3,Seis lesionados en choque entre un taxi y un a...,"Rafael Lucio, Ver. 01/01/2024",Desconocido,Seis lesionados dejó esta noche el choque entr...,https://www.alcalorpolitico.com/informacion/se...,1063,"Rafael Lucio, Ver",01/01/2024
4,Limpian bulevar Manuel Ávila Camacho en Boca d...,01/01/2024,Desconocido,Con el “barrido” sobre el bulevar Manuel Ávila...,https://www.alcalorpolitico.com/informacion/li...,374,NaN,01/01/2024
...,...,...,...,...,...,...,...,...
23394,Asesinan a balazos a taquero en la colonia El ...,"Córdoba, Ver. 20/04/2025",Desconocido,"Un taquero identificado como José Enrique ""N"",...",https://www.alcalorpolitico.com/informacion/as...,1523,"Córdoba, Ver",20/04/2025
23395,"Domingo caluroso; posibles tormentas, lluvias ...","Xalapa, Ver. 20/04/2025",Desconocido,"Probabilidad de lluvias, chubascos y tormentas...",https://www.alcalorpolitico.com/informacion/do...,2103,"Xalapa, Ver",20/04/2025
23396,La Pascua es el corazón del año litúrgico y el...,"Xalapa, Ver. 20/04/2025",Desconocido,"Este Domingo de Resurrección, el mensaje que r...",https://www.alcalorpolitico.com/informacion/la...,1613,"Xalapa, Ver",20/04/2025
23397,Católicos celebran el Domingo de Resurrección,"Xalapa, Ver. 20/04/2025",Desconocido,"Este 20 de abril, la comunidad católica celebr...",https://www.alcalorpolitico.com/informacion/ca...,1551,"Xalapa, Ver",20/04/2025


In [25]:
# Eliminar duplicados (conservando primeros)
#voz.drop_duplicates(inplace=True)
#letra.drop_duplicates(inplace=True)
calor.drop_duplicates(inplace=True)
#plumas.drop_duplicates(inplace=True)

In [23]:
calor

,titulo,fecha,autor,contenido,url,longitud,lugar,fecha_limpia
0,Magistrada Mónica Soto asume como presidenta d...,Ciudad de México 01/01/2024,Desconocido,La magistrada Mónica Soto inició este 1° de en...,https://www.alcalorpolitico.com/informacion/ma...,2173,Ciudad de México,01/01/2024
1,Instructora de Orizaba lamenta “desvalorizació...,"Orizaba, Ver. 01/01/2024",Desconocido,Aunque a través de los talleres del DIF se bus...,https://www.alcalorpolitico.com/informacion/in...,1494,"Orizaba, Ver",01/01/2024
2,"Hacen “sándwich” a auto particular, en autopis...",Fortín de las Flores 01/01/2024,Desconocido,Daños materiales por varios miles de pesos y l...,https://www.alcalorpolitico.com/informacion/ha...,1050,Fortín de las Flores,01/01/2024
3,Seis lesionados en choque entre un taxi y un a...,"Rafael Lucio, Ver. 01/01/2024",Desconocido,Seis lesionados dejó esta noche el choque entr...,https://www.alcalorpolitico.com/informacion/se...,1063,"Rafael Lucio, Ver",01/01/2024
4,Limpian bulevar Manuel Ávila Camacho en Boca d...,01/01/2024,Desconocido,Con el “barrido” sobre el bulevar Manuel Ávila...,https://www.alcalorpolitico.com/informacion/li...,374,NaN,01/01/2024
...,...,...,...,...,...,...,...,...
23394,Asesinan a balazos a taquero en la colonia El ...,"Córdoba, Ver. 20/04/2025",Desconocido,"Un taquero identificado como José Enrique ""N"",...",https://www.alcalorpolitico.com/informacion/as...,1523,"Córdoba, Ver",20/04/2025
23395,"Domingo caluroso; posibles tormentas, lluvias ...","Xalapa, Ver. 20/04/2025",Desconocido,"Probabilidad de lluvias, chubascos y tormentas...",https://www.alcalorpolitico.com/informacion/do...,2103,"Xalapa, Ver",20/04/2025
23396,La Pascua es el corazón del año litúrgico y el...,"Xalapa, Ver. 20/04/2025",Desconocido,"Este Domingo de Resurrección, el mensaje que r...",https://www.alcalorpolitico.com/informacion/la...,1613,"Xalapa, Ver",20/04/2025
23397,Católicos celebran el Domingo de Resurrección,"Xalapa, Ver. 20/04/2025",Desconocido,"Este 20 de abril, la comunidad católica celebr...",https://www.alcalorpolitico.com/informacion/ca...,1551,"Xalapa, Ver",20/04/2025


### Script para limpiar database de La Voz del Sur

In [42]:
# Función para extraer los tags y limpiar el artículo
def procesar_articulo(texto):
    # Buscar los tags (ejemplo: "Tags: Chava Zamora, jalisco, ...")
    tags_match = re.search(r'Tags:\s*(.*?)(?=\n\n|$)', texto, flags=re.DOTALL | re.IGNORECASE)
    tags = tags_match.group(1).strip() if tags_match else ""

    # Eliminar la línea de tags del artículo
    texto_limpio = re.sub(r'Tags:.*?(?=\n\n|$)', '', texto, flags=re.DOTALL | re.IGNORECASE)

    # Limpieza adicional (lo que ya teníamos antes)
    texto_limpio = re.sub(r'Published on.*?by lavozsur', '', texto_limpio, flags=re.DOTALL)
    texto_limpio = re.sub(r'lavozsur Con 35 años de trayectoria.*?Jalisco\.', '', texto_limpio, flags=re.DOTALL)
    texto_limpio = re.sub(r'Comments are closed\.', '', texto_limpio, flags=re.DOTALL)
    texto_limpio = texto_limpio.strip()

    return texto_limpio, tags

# Aplicar la función y crear las nuevas columnas
voz[['articulo', 'tags']] = pd.DataFrame(voz['articulo'].apply(procesar_articulo).tolist(), index=voz.index)


# Función para extraer el lugar y limpiar el artículo
def extraer_lugar(texto):
    # Patrón para ciudades en mayúsculas seguidas de "JAL." o similares
    lugar_match = re.search(r'^([A-ZÁÉÍÓÚÜÑ\s]+(?:,\s*JAL\.?|\.))', texto)
    if lugar_match:
        lugar = lugar_match.group(1).strip()
        texto_limpio = texto.replace(lugar, '', 1).strip()
    else:
        lugar = ""
        texto_limpio = texto
    return texto_limpio, lugar

# Aplicar la función para extraer lugares
voz[['articulo', 'lugar']] = pd.DataFrame(voz['articulo'].apply(extraer_lugar).tolist(), index=voz.index)

# Función para verificar si contiene (ES)
def contiene_ES(texto):
    return 'yes' if '(ES)' in texto else 'no'

# Crear columna 'contiene_ES'
voz['contiene_ES'] = voz['articulo'].apply(contiene_ES)

# Reordenar columnas 
column_order = ['titulo', 'articulo', 'lugar', 'tags', 'contiene_ES', 'fecha', 'url']
voz = voz[column_order]

# Guardar el resultado (opcional)
# voz.to_csv('cleaned_vozdelsur.csv', index=False, encoding='utf-8-sig')

### Script para limpiar database de Letra Fría

In [43]:
# Función para extraer los bylines y limpiar el artículo
def procesar_articulo(texto):
    # Buscar los bylines (ejemplo: "Por: Lauro Rodríguez, 	Por: Vianney Martínez Pérez, etc")
    tags_match = re.search(r'Por:\s*(.*?)(?=\n\n|$)', texto, flags=re.DOTALL | re.IGNORECASE)
    tags = tags_match.group(1).strip() if tags_match else ""

    # Eliminar la línea de tags del artículo
    texto_limpio = re.sub(r'Por:.*?(?=\n\n|$)', '', texto, flags=re.DOTALL | re.IGNORECASE)

    # Limpieza adicional
    texto_limpio = re.sub(r'Suscríbete a nuestro WhatsApp y recibe las noticias todos los días\.', '', texto_limpio, flags=re.DOTALL)
    texto_limpio = texto_limpio.strip()

    return texto_limpio, tags

# Aplicar la función y crear las nuevas columnas
letra = letra.copy()
letra[['articulo', 'autor']] = pd.DataFrame(letra['articulo'].apply(procesar_articulo).tolist(), index=letra.index)

# Función para extraer el lugar
def extraer_lugar(texto):
    # Patrón más flexible para nombres de ciudad (1-5 palabras) seguidas de "Jalisco.-"
    lugar_match = re.search(r'((?:\b[\wáéíóúüñÁÉÍÓÚÜÑ]+\b\s*){1,5}, Jalisco\.-)', texto)
    if lugar_match:
        lugar = lugar_match.group(1).strip()
        # Eliminar ".-" al final y cualquier espacio sobrante
        lugar = re.sub(r'\.-\s*$', '', lugar)
        texto_limpio = texto.replace(lugar_match.group(0), '', 1).strip()
        # Limpieza adicional: eliminar espacios dobles que puedan quedar
        texto_limpio = re.sub(r'\s+', ' ', texto_limpio).strip()
    else:
        lugar = ""
        texto_limpio = texto
    return texto_limpio, lugar

# Aplicar la función para extraer lugares
letra[['articulo', 'lugar']] = pd.DataFrame(letra['articulo'].apply(extraer_lugar).tolist(), index=letra.index)

# Reordenar columnas (opcional)
column_order = ['titulo', 'articulo', 'lugar', 'autor', 'fecha', 'url']
letra = letra[column_order]

# Guardar el resultado (opcional)
# letra.to_csv('cleaned_letrafria.csv', index=False, encoding='utf-8-sig')

In [44]:
#Función para sacar muestra estratificada de Al Calor Político porque es un database muy grande.


# Asumiendo que tu DataFrame se llama 'dfcalor' y tiene columna 'fecha_limpia'
# Convertimos a datetime para facilitar el manejo
calor['fecha_limpia'] = pd.to_datetime(calor['fecha_limpia'], errors='coerce')

# Eliminamos filas con fechas inválidas (por si acaso)
calor = calor.dropna(subset=['fecha_limpia'])

# Función para obtener muestra estratificada
def muestra_estratificada(df, fecha_col='fecha_limpia', muestras_por_dia=7):
    # Extraer año-mes-dia como columna separada
    df['fecha_str'] = df[fecha_col].dt.strftime('%Y-%m-%d')
    
    # Agrupar por día y muestrear
    muestra = (df.groupby('fecha_str', group_keys=False)
              .apply(lambda x: x.sample(min(len(x), muestras_por_dia), random_state=1)))
    
    return muestra.drop(columns=['fecha_str'])

# Aplicar la función
muestra_calor = muestra_estratificada(calor)

# Verificación
print(f"📊 Muestra obtenida: {len(muestra_calor)} artículos")
print("Distribución por mes:")
print(muestra_calor['fecha_limpia'].dt.to_period('M').value_counts().sort_index())

# Guardar como nuevo CSV
muestra_calor.to_csv('muestra_estratificada_aldfcalor.csv', index=False, encoding='utf-8-sig')

/tmp/ipykernel_9483/2765170283.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(min(len(x), muestras_por_dia), random_state=1)))


📊 Muestra obtenida: 1344 artículos
Distribución por mes:
fecha_limpia
2024-01    84
2024-02    84
2024-03    84
2024-04    84
2024-05    84
2024-06    84
2024-07    84
2024-08    84
2024-09    84
2024-10    84
2024-11    84
2024-12    84
2025-01    28
2025-02    28
2025-03    28
2025-04    28
2025-05    28
2025-06    28
2025-07    28
2025-08    28
2025-09    28
2025-10    28
2025-11    28
2025-12    28
Freq: M, Name: count, dtype: int64


### Análisis de contenido

In [9]:
plumas['articulo'] = plumas['articulo'].astype("string")

/tmp/ipykernel_1790/3210223941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plumas['articulo'] = plumas['articulo'].astype("string")


In [10]:
# Filtrar filas donde 'articulo' no es nulo
plumas = plumas.dropna(subset=['articulo'])

In [11]:
# Establecer función para analizar la base de datos
def analizar_articulo(texto):
    doc = nlp(texto)

    # 1. Entidades nombradas (ej: lugares, personas)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]

    # 2. Sustantivos y adjetivos (para temas clave)
    palabras_clave = [token.text for token in doc if token.pos_ in ["NOUN", "ADJ"]]

    # 3. Lematización (reducir las variants de una palabra a su forma básica)
    lemas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    # 4. Frecuencia de términos
    frecuencias = Counter([token.text.lower() for token in doc if not token.is_stop and token.is_alpha])

    return {
        "entidades": entidades,
        "palabras_clave": palabras_clave[:10],  # Top 10
        "lemas": lemas[:20],  # Primeros 20 lemas
        "frecuencias": frecuencias.most_common(5)  # Top 5 palabras
    }

# Aplicar a la columna 'articulo' o 'contenido' de cada base de datos
#voz["analisis"] = voz["articulo"].apply(analizar_articulo)
#letra["analisis"] = letra["articulo"].apply(analizar_articulo)
#muestra_calor["analisis"] = muestra_calor["contenido"].apply(analizar_articulo)
plumas["analisis"] = plumas["articulo"].apply(analizar_articulo)

# Ejemplo de acceso a los resultados
#print(voz["analisis"])  # Ver análisis del primer artículo
#print(letra["analisis"])  # Ver análisis del primer artículo
#print(muestra_calor["analisis"])  # Ver análisis del primer artículo
print(plumas["analisis"])  # Ver análisis del primer artículo

0      {'entidades': [('Veracruz', 'LOC'), ('México',...
1      {'entidades': [('Paco De Luna', 'PER'), ('Xala...
2      {'entidades': [('Efraín', 'LOC'), ('delPaso Su...
3      {'entidades': [('Luis Ortiz', 'PER'), ('Zócalo...
4      {'entidades': [('Cdmx.- Tres hombres y dos muj...
                             ...                        
465    {'entidades': [('Manuela V. CarmonaIxtaczoquit...
466    {'entidades': [('Xalapa', 'LOC'), ('El frente ...
467    {'entidades': [('Pascacio Taboada Cortina', 'P...
468    {'entidades': [('Xalapa', 'LOC'), ('Ver.-', 'L...
469    {'entidades': [('CDMX.- La Confederación Patro...
Name: analisis, Length: 469, dtype: object


### Análisis de sentimientos

In [12]:
# 1. Cargar el modelo (se descarga la primera vez)
classifier = pipeline(
    "text-classification",
    model="finiteautomata/beto-sentiment-analysis",
    tokenizer="finiteautomata/beto-sentiment-analysis"
)

# 2. Función para clasificar 
def clasificar_sentimiento(texto):
    chunks = [texto[i:i+512] for i in range(0, len(texto), 512)]  # Divide en trozos de 512 tokens para manejar textos largos
    resultados = [classifier(chunk)[0]['label'] for chunk in chunks]
    # Cuenta votos (si hay trozos con distintos sentimientos)
    return max(set(resultados), key=resultados.count)

# 3. Aplicar al DataFrame (puede tardar en datasets grandes)
#voz['sentimiento'] = voz['articulo'].apply(clasificar_sentimiento)
#letra['sentimiento'] = letra['articulo'].apply(clasificar_sentimiento)
#muestra_calor['sentimiento'] = muestra_calor['contenido'].apply(clasificar_sentimiento)
plumas['sentimiento'] = plumas['articulo'].apply(clasificar_sentimiento)


# Ver resultados
#print(voz[['articulo', 'sentimiento']].head())
# Ver resultados
#print(letra[['articulo', 'sentimiento']].head())
# Ver resultados
#print(muestra_calor[['contenido', 'sentimiento']].head())
# Ver resultados
print(plumas[['articulo', 'sentimiento']].head())

Device set to use cpu


                                            articulo sentimiento
0  Veracruz, Ver. En México, la justicia sigue fa...         NEG
1  Paco De Luna /
Xalapa, Ver.-Debido a la toma d...         NEG
2  Por Efraín Hernández/
Río Blanco.- La cancelac...         NEG
3  Luis Ortiz
En el Zócalo de la ciudad de Veracr...         NEU
4  Cdmx.- Tres hombres y dos mujeres fueron asesi...         NEG


In [13]:
# Guardar el resultado
#voz.to_csv('analisis_vozdelsur.csv', index=False, encoding='utf-8-sig')
#letra.to_csv('analisis_letrafria.csv', index=False, encoding='utf-8-sig')
#muestra_calor.to_csv('analisis_alcalorpolitico.csv', index=False, encoding='utf-8-sig')
plumas.to_csv('analisis_plumaslibres.csv', index=False, encoding='utf-8-sig')


In [14]:
plumas

,url,titulo,articulo,fecha,autor,analisis,sentimiento
0,https://plumaslibres.com.mx/2025/03/20/victima...,"Víctima de violencia, condenada por secuestro:...","Veracruz, Ver. En México, la justicia sigue fa...",20 Mar 25,"Veracruz, Ver. En México, la justicia sigue fa...","{'entidades': [('Veracruz', 'LOC'), ('México',...",NEG
1,https://plumaslibres.com.mx/2025/02/25/union-d...,Unión de sindicatos se oponen a entrega de pla...,"Paco De Luna / Xalapa, Ver.-Debido a la toma d...",25 Feb 25,Paco De Luna /,"{'entidades': [('Paco De Luna', 'PER'), ('Xala...",NEG
2,https://plumaslibres.com.mx/2025/04/11/fracasa...,Fracasa alcalde morenista de Río Blanco Ricard...,Por Efraín Hernández/ Río Blanco.- La cancelac...,11 Abr 25,Efraín Hernández/,"{'entidades': [('Efraín', 'LOC'), ('delPaso Su...",NEG
3,https://plumaslibres.com.mx/2025/03/08/colocan...,Colocan corazón con listones morados en el Zóc...,Luis Ortiz En el Zócalo de la ciudad de Veracr...,8 Mar 25,Luis Ortiz,"{'entidades': [('Luis Ortiz', 'PER'), ('Zócalo...",NEU
4,https://plumaslibres.com.mx/2025/02/10/masacre...,"Masacre en CDMX matan a 5 en Tláhuac, violenci...",Cdmx.- Tres hombres y dos mujeres fueron asesi...,10 Feb 25,Cdmx.- Tres hombres y dos mujeres fueron asesi...,{'entidades': [('Cdmx.- Tres hombres y dos muj...,NEG
...,...,...,...,...,...,...,...
465,https://plumaslibres.com.mx/2024/01/11/protest...,Protestan en Ixtaczoquitlán porque el Ayuntami...,"Manuela V. CarmonaIxtaczoquitlán, Ver.- Habita...",11 Ene 24,"Manuela V. CarmonaIxtaczoquitlán, Ver.- Habita...",{'entidades': [('Manuela V. CarmonaIxtaczoquit...,NEG
466,https://plumaslibres.com.mx/2024/01/12/preven-...,Prevén hoy Surada en el sur y la masa de aire ...,"Xalapa, Ver.El frente frío 26 recorre con rapi...",12 Ene 24,"Xalapa, Ver.El frente frío 26 recorre con rapi...","{'entidades': [('Xalapa', 'LOC'), ('El frente ...",NEU
467,https://plumaslibres.com.mx/2024/02/06/por-efe...,"Por efecto de la pandemia en México, 250 mil n...",Pascacio Taboada Cortina/Jorge Martínez Cedill...,6 Feb 24,Pascacio Taboada Cortina/Jorge Martínez Cedillo,"{'entidades': [('Pascacio Taboada Cortina', 'P...",NEG
468,https://plumaslibres.com.mx/2024/01/27/tramita...,Tramita amparo ex vocera de Javier Duarte para...,"Xalapa, Ver.- Con fecha 23 de enero de este añ...",27 Ene 24,"Xalapa, Ver.- Con fecha 23 de enero de este añ...","{'entidades': [('Xalapa', 'LOC'), ('Ver.-', 'L...",NEU


In [16]:
# Buscador de palabras clave

# Palabras clave a buscar (puedes añadir sinónimos o variantes)
palabras_clave = [
    "narco", 
    "narcotráfico", 
    "desaparecido", 
    "narcotraficante"
]

# Función para verificar si un artículo contiene ALGUNA de las palabras clave
def contiene_narco(texto):
    texto_lower = str(texto).lower()  # Convertir a string y minúsculas
    return any(keyword in texto_lower for keyword in palabras_clave)

# Aplicar la función a la columna 'articulo' y sumar los TRUE
conteo_ddhh = plumas['articulo'].apply(contiene_narco).sum()

print(f"Número de artículos sobre narco: {conteo_ddhh} de {len(plumas)}")

Número de artículos sobre narco: 45 de 469


In [18]:

# Patrón regex: busca "derechos humanos" y variantes, evitando otros contextos
patron = r"\bderechos\s*humanos\b|\bdd\.?hh\.?\b|\bderecho\s*humano\b"

def conteo_regex(texto):
    return bool(re.search(patron, str(texto).lower(), flags=re.IGNORECASE))

conteo_preciso = plumas['articulo'].apply(conteo_regex).sum()

print(f"Artículos sobre derechos humanos (regex): {conteo_preciso}")

Artículos sobre derechos humanos (regex): 26
